# Implementation of ACGAN Model
*by Marvin Bertin*
<img src="../../images/keras-tensorflow-logo.jpg" width="400">

# Imports

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow as tf
import numpy as np

In [2]:
models = tf.contrib.keras.models
layers = tf.contrib.keras.layers
utils = tf.contrib.keras.utils
losses = tf.contrib.keras.losses
optimizers = tf.contrib.keras.optimizers 
metrics = tf.contrib.keras.metrics

# Construct Generator
<img src="../../images/acgan-3.png" width="800">

In [11]:
def generator(latent_size, classes=10):
    
    def up_sampling_block(x, filter_size):
        # upsample block
        x = layers.UpSampling2D(size=(2,2))(x)
        x = layers.Conv2D(filter_size, (5,5), padding='same', activation='relu')(x)
        
        return x
    
    # Input 1
    # image class label
    image_class = layers.Input(shape=(1,), dtype='int32', name='image_class')
    
    # class embeddings
    emb = layers.Embedding(classes, latent_size,
                           embeddings_initializer='glorot_normal')(image_class)
    
    # 10 classes in MNIST
    cls = layers.Flatten()(emb)
    
    # Input 2
    # latent noise vector
    latent_input = layers.Input(shape=(latent_size,), name='latent_noise')
    
    # hadamard product between latent embedding and a class conditional embedding
    h = layers.multiply([latent_input, cls])
    
    # Conv generator
    x = layers.Dense(1024, activation='relu')(h)
    x = layers.Dense(128 * 7 * 7, activation='relu')(x)
    x = layers.Reshape((7, 7, 128))(x)
    
    # upsample to (14, 14, 128)
    x = up_sampling_block(x, 128)
    
    # upsample to (28, 28, 256)
    x = up_sampling_block(x, 256)
    
    # reduce channel into binary image (28, 28, 1)
    generated_img = layers.Conv2D(1, (2,2), padding='same', activation='tanh')(x)
    
    return models.Model(inputs=[latent_input, image_class],
                        outputs=generated_img,
                        name='generator') 

In [23]:
g = generator(latent_size = 100, classes=10)
g.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
image_class (InputLayer)         (None, 1)             0                                            
____________________________________________________________________________________________________
embedding_6 (Embedding)          (None, 1, 100)        1000        image_class[0][0]                
____________________________________________________________________________________________________
latent_noise (InputLayer)        (None, 100)           0                                            
____________________________________________________________________________________________________
flatten_10 (Flatten)             (None, 100)           0           embedding_6[0][0]                
___________________________________________________________________________________________

# Construct Discriminator
<img src="../../images/acgan-3.png" width="800">

In [15]:
def discriminator(input_shape=(28, 28, 1)):
    
    def conv_block(x, filter_size, stride):
        
        x = layers.Conv2D(filter_size, (3,3), padding='same', strides=stride)(x)
        x = layers.LeakyReLU()(x)
        x = layers.Dropout(0.3)(x)
        return x
    
    input_img = layers.Input(shape=input_shape)
    
    # discriminator network
    x = conv_block(input_img, 32, (2,2))
    x = conv_block(input_img, 64, (1,1))
    x = conv_block(input_img, 128, (2,2))
    x = conv_block(input_img, 256, (1,1))
    
    features = layers.Flatten()(x)
    
    # binary classifier, image fake or real
    fake = layers.Dense(1, activation='sigmoid', name='generation')(features)
    
    # multi-class classifier, image digit class
    aux = layers.Dense(10, activation='softmax', name='auxiliary')(features)
    
    
    return models.Model(inputs=input_img, outputs=[fake, aux], name='discriminator')

In [24]:
d = discriminator(input_shape=(28, 28, 1))
d.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_6 (InputLayer)             (None, 28, 28, 1)     0                                            
____________________________________________________________________________________________________
conv2d_35 (Conv2D)               (None, 14, 14, 32)    320         input_6[0][0]                    
____________________________________________________________________________________________________
leaky_re_lu_17 (LeakyReLU)       (None, 14, 14, 32)    0           conv2d_35[0][0]                  
____________________________________________________________________________________________________
dropout_17 (Dropout)             (None, 14, 14, 32)    0           leaky_re_lu_17[0][0]             
___________________________________________________________________________________________

# Combine Generator with Discriminator
<img src="../../images/acgan-2.png" width="300">

In [17]:
# Adam parameters suggested in paper
adam_lr = 0.0002
adam_beta_1 = 0.5

def ACGAN(latent_size = 100):
    # build the discriminator
    dis = discriminator()
    dis.compile(
        optimizer=optimizers.Adam(lr=adam_lr, beta_1=adam_beta_1),
        loss=['binary_crossentropy', 'sparse_categorical_crossentropy']
    )

    # build the generator
    gen = generator(latent_size)
    gen.compile(optimizer=optimizers.Adam(lr=adam_lr, beta_1=adam_beta_1),
                      loss='binary_crossentropy')

    # Inputs
    latent = layers.Input(shape=(latent_size, ), name='latent_noise')
    image_class = layers.Input(shape=(1,), dtype='int32', name='image_class')

    # Get a fake image
    fake_img = gen([latent, image_class])

    # Only train generator in combined model
    dis.trainable = False
    fake, aux = dis(fake_img)
    combined = models.Model(inputs=[latent, image_class],
                            outputs=[fake, aux],
                            name='ACGAN')

    combined.compile(
        optimizer=optimizers.Adam(lr=adam_lr, beta_1=adam_beta_1),
        loss=['binary_crossentropy', 'sparse_categorical_crossentropy']
    )
    
    return combined

In [18]:
cagan = ACGAN(latent_size = 100)
cagan.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
latent_noise (InputLayer)        (None, 100)           0                                            
____________________________________________________________________________________________________
image_class (InputLayer)         (None, 1)             0                                            
____________________________________________________________________________________________________
generator (Model)                (None, 28, 28, 1)     8172521     latent_noise[0][0]               
                                                                   image_class[0][0]                
____________________________________________________________________________________________________
discriminator (Model)            [(None, 1), (None, 10 525835      generator[1][0]         

## Next Lesson
### Train and Evaluate ACGAN
- Image classification task with MNIST

<img src="../../images/divider.png" width="100">